- Update: 2023.02.11

# 0. 配置参数

- `dict_menu`：目录文件的路径
- `out`：保存结果的路径

In [1]:
dict_menu = {'file': '../../file/cardone-wiper_motor-menu.xlsx',
             'head': 1}

out = '../../file/cardone-wiper_motor-content.xlsx'

In [2]:
import pandas as pd

In [3]:
df_menu = pd.read_excel(dict_menu['file'], header=dict_menu['head']-1)

df_menu

,标题,品牌号,车型,链接
0,Windshield Wiper Motor - 40-1000,40-1000,Chevrolet Camaro 1998-93\nPontiac Firebird 199...,https://www.cardone.com/product/windshield-wip...
1,Windshield Wiper Motor - 40-10000,40-10000,Hummer H3 2010-06,https://www.cardone.com/product/windshield-wip...
2,Windshield Wiper Motor - 40-10005,40-10005,"Cadillac CTS 2015-08, Escalade 2022-15, Escala...",https://www.cardone.com/product/windshield-wip...
3,Windshield Wiper Motor - 40-10006,40-10006,Buick Enclave 2017-08,https://www.cardone.com/product/windshield-wip...
4,Windshield Wiper Motor - 40-10007,40-10007,Chevrolet Malibu 2007-05,https://www.cardone.com/product/windshield-wip...
5,Windshield Wiper Motor - 40-10009,40-10009,"Chevrolet Suburban 1500 2000, Suburban 2500 20...",https://www.cardone.com/product/windshield-wip...


In [4]:
from selenium.webdriver import Chrome
import datetime

In [5]:
browser = Chrome()
browser.set_page_load_timeout(60)

print(f'开始爬取时间：{datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")}')

开始爬取时间：2023:02:11 08:54:04


In [6]:
from selenium.webdriver.common.by import By
import time

In [7]:
col_href = []
col_oe = []
for i in range(len(df_menu)):
    print(i+1)
    browser.get(df_menu.loc[i, '链接'] + '#tab-specs')
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1.5)
    browser.execute_script('window.scrollTo(0, -document.body.scrollHeight)')
    time.sleep(0.5)
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    
    href = browser.find_element(by=By.CLASS_NAME, value='productView-img-container').find_element(by=By.TAG_NAME, value='a').get_attribute('href')
    col_href.append(href)
    # print(href)
    list_oe = []
    list_div = browser.find_element(by=By.CLASS_NAME, value='oe-nums').find_elements(by=By.CLASS_NAME, value='productView-table-data')
    for div in list_div:
        list_str = div.text.split(',')
        for string in list_str:
            list_oe.append(string.strip())
    list_oe = set(list_oe)
    oe = ';'.join(list_oe)
    col_oe.append(oe)
    # print(oe)
    # print()

1
2
3
4
5
6


In [8]:
df_menu['图片链接'] = col_href
df_menu['OE号'] = col_oe

df_content = df_menu[['标题', '品牌号', 'OE号', '车型', '链接', '图片链接']]

In [9]:
browser.quit()
print(f'爬取结束时间：{datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")}')

爬取结束时间：2023:02:11 08:54:37


In [10]:
df_content.to_excel(out, index=False)